<p style="text-align:right;">
<img src="TimberTech.png"
     style="float: center; margin-right: 10px;"
     width="200"
     />
</p>
<h1 style="text-align: center;">Swan Consulting</h1>
<h2 style="text-align: center;">Predicting Churners</h2>
<h3 style="text-align: center;">By Timber Tech</h3>
<h4 style="text-align: center;">Georgia, Iqra, Zach</h4>


The goal of this notebook is to load in the Swan Dataset, apply some feature engineering to the variables and build a series of classification models to predict churn risk. Once we have developed a suitable model, we will create tables for the probability of each person churning, as well as providing the 500 customers most likely to churn.

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

## 1. Data Import and Exploration

In [2]:
swan = pd.read_excel("1 - Project Data.xlsx")
swan = swan.copy()
swan.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices


First, we check for nulls and datatypes:

In [3]:
swan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

## 2. Feature Engineering

Before we can start fitting models, we need to encode the variables. Many of them are in the form Yes or No, which we convert to 1 and 0, and the contract, payment method and internet serivce variables can be one hot encoded.

In [4]:
def yes_no(column):
    x=column
    if x=='Yes':
        return 1
    else:
        return 0

swan=swan[swan['Total Charges']!=' '] #Some customers are in the database but never actually used the product, also have Tenure=0

Now we can combine this all into one feature engineering function:

In [5]:
def feature_eng(x):
    df=x.copy()
    #Apply the yes-no encoding to the following columns
    yes_no_columns=['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines',
                    'Online Security', 'Online Backup', 'Device Protection', 
                    'Tech Support', 'Streaming TV', 'Streaming Movies', 'Paperless Billing']
    for column in yes_no_columns:
        df[column]=df[column].apply(yes_no) 
    #Apply OHE for the following columns
    encoding_columns=['Contract','Payment Method', 'Internet Service']
    encoding_prefix=['Contract','Payment', 'Internet']
    df= pd.get_dummies(data=df, 
                       columns=encoding_columns, 
                       prefix=encoding_prefix, 
                       dtype=int)
    #Map male and female to numerical values
    df.Gender = df.Gender.map({'Male':0, 'Female':1})

    return df

Now we can check that all the variables are numerical.

In [6]:
feature_eng(swan).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   CustomerID                         7032 non-null   object 
 1   Count                              7032 non-null   int64  
 2   Country                            7032 non-null   object 
 3   State                              7032 non-null   object 
 4   City                               7032 non-null   object 
 5   Zip Code                           7032 non-null   int64  
 6   Lat Long                           7032 non-null   object 
 7   Latitude                           7032 non-null   float64
 8   Longitude                          7032 non-null   float64
 9   Gender                             7032 non-null   int64  
 10  Senior Citizen                     7032 non-null   int64  
 11  Partner                            7032 non-null   int64

Before we can start fitting models, it is worth noting that there is a massive class imbalance in the data. There are approximately 3 times as many non-churners as churners, and so we need to employ some resampling methods in order to fit a robust model. We used a model called SMOTE (Synthetic Minority Oversampling Technique), which will approximately balance churners and non-churners. We need to apply this after the test-train split, in order to prevent data leakage.

In [7]:
feature_cols=[
            'Gender','Senior Citizen','Partner',
            'Dependents','Tenure Months','Phone Service',
            'Multiple Lines','Online Security','Online Backup',
            'Device Protection','Tech Support','Streaming TV',
            'Streaming Movies','Paperless Billing','Monthly Charges',
            'Total Charges','Contract_Month-to-month','Contract_One year',
            'Contract_Two year','Payment_Bank transfer (automatic)',
            'Payment_Credit card (automatic)','Payment_Electronic check',
            'Payment_Mailed check','Internet_DSL','Internet_Fiber optic',
            'Internet_No']

target='Churn Value'

X_train, X_test, y_train, y_test = train_test_split(feature_eng(swan)[feature_cols], swan[target], test_size = 0.3, random_state = 42)


smote=SMOTE()
X_train, y_train = smote.fit_resample(X_train,y_train)

## 3. Model Fit and Evaluation

We are going to consider three different classification models for predicting churn risk: Logistic Regression, Random Forest Classifier and Extra Trees Classifier. First of all, we are going to create baseline models for each of the three, using all of the available variables and limited tuning of the hyper parameters. Based on the performance of these, we can decide which model we want to use.

In [8]:
threshold=0.5 
#We will start with a threshold prediction of 0.5, which we can change later

In [9]:
lr = LogisticRegression(max_iter=1000, 
                        random_state=10
                       )
lr.fit(X_train, y_train)

lr_train_prob=lr.predict_proba(X_train)[:,1]
lr_train_pred=np.where(lr_train_prob>threshold,1,0)
print(metrics.classification_report(lr_train_pred, y_train))
print("")
lr_test_prob=lr.predict_proba(X_test)[:,1]
lr_test_pred=np.where(lr_test_prob>threshold,1,0)
print(metrics.classification_report(lr_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84      3532
           1       0.86      0.84      0.85      3750

    accuracy                           0.85      7282
   macro avg       0.85      0.85      0.85      7282
weighted avg       0.85      0.85      0.85      7282


              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1458
           1       0.70      0.63      0.66       652

    accuracy                           0.80      2110
   macro avg       0.77      0.75      0.76      2110
weighted avg       0.80      0.80      0.80      2110



In [10]:
#At this point, we have done very little tuning of the hyper parameters
#This model is just to illustrate the general performance
rf = RandomForestClassifier(n_estimators=150, 
                            max_depth=5, 
                            min_samples_split=2, 
                            criterion='gini')
rf.fit(X_train,y_train)
rf_train_prob=rf.predict_proba(X_train)[:,1]
rf_train_pred= np.where(rf_train_prob>threshold, 1, 0)
print(metrics.classification_report(rf_train_pred, y_train))

rf_test_prob=rf.predict_proba(X_test)[:,1]
rf_test_pred=np.where(rf_test_prob>threshold,1,0)
print(metrics.classification_report(rf_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.76      0.89      0.82      3108
           1       0.90      0.79      0.84      4174

    accuracy                           0.83      7282
   macro avg       0.83      0.84      0.83      7282
weighted avg       0.84      0.83      0.83      7282

              precision    recall  f1-score   support

           0       0.76      0.91      0.83      1278
           1       0.80      0.56      0.66       832

    accuracy                           0.77      2110
   macro avg       0.78      0.74      0.74      2110
weighted avg       0.78      0.77      0.76      2110



In [11]:
et = ExtraTreesClassifier(n_estimators=150, 
                          max_depth=5, 
                          min_samples_split=2)
et.fit(X_train,y_train)
et_train_prob=et.predict_proba(X_train)[:,1]
et_train_pred= np.where(et_train_prob>threshold, 1, 0)
print(metrics.classification_report(et_train_pred, y_train))
print("")
et_test_prob=et.predict_proba(X_test)[:,1]
et_test_pred= np.where(et_test_prob>threshold, 1, 0)
print(metrics.classification_report(et_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80      3075
           1       0.90      0.77      0.83      4207

    accuracy                           0.82      7282
   macro avg       0.82      0.83      0.82      7282
weighted avg       0.83      0.82      0.82      7282


              precision    recall  f1-score   support

           0       0.75      0.90      0.82      1257
           1       0.80      0.55      0.65       853

    accuracy                           0.76      2110
   macro avg       0.77      0.73      0.73      2110
weighted avg       0.77      0.76      0.75      2110



In these three models, we have elected to consider all of variables in the dataset. Based on these initial evaluations, we are not going to consider extra trees classifier, as this seems have lower accuracy, and is similar in nature to Random Forests.

## 4. Optimisation of Random Forest

Before we decide which model we want to use, we are going to tune the hyper-parameters of the random forests model. To do so, we implement a grid search.

In [12]:
rf=RandomForestClassifier(n_estimators=50)
# We have decided to optimise over the following parameters
rf_params = {
    'max_depth': [2, 4, 6, 8, 10], #Depth of trees
    'min_samples_split': [2, 4, 6, 8, 10], # Minimum number of samples required for a split
    'criterion': ['gini', 'entropy'], # Splitting criterion
    'class_weight':['balanced', None] # Rebalancing criterion
    
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=5, verbose=1)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_
#Note, grid search returns slightly different parameters each time. We ran multiple times and selected the most freuqent parameters

Fitting 5 folds for each of 100 candidates, totalling 500 fits
0.8392116119981596


{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_split': 2}

In [28]:
#Model using optimised parameters
threshold=0.5
rf = RandomForestClassifier(n_estimators=250, 
                            max_depth=10, 
                            min_samples_split=2, 
                            criterion='entropy',
                            class_weight=None
                           )
rf.fit(X_train,y_train)
rf_train_prob=rf.predict_proba(X_train)[:,1]
rf_train_pred= np.where(rf_train_prob>threshold, 1, 0)
print(metrics.classification_report(rf_train_pred, y_train))
print("")
rf_test_prob=rf.predict_proba(X_test)[:,1]
rf_test_pred=np.where(rf_test_prob>threshold,1,0)
print(metrics.classification_report(rf_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89      3270
           1       0.95      0.86      0.90      4012

    accuracy                           0.90      7282
   macro avg       0.90      0.90      0.89      7282
weighted avg       0.90      0.90      0.90      7282


              precision    recall  f1-score   support

           0       0.81      0.89      0.85      1391
           1       0.74      0.61      0.67       719

    accuracy                           0.79      2110
   macro avg       0.78      0.75      0.76      2110
weighted avg       0.79      0.79      0.79      2110



After optimising, the model seems to overfit, so we increase the threshold for positive prediction.

In [32]:
threshold=0.8
rf = RandomForestClassifier(n_estimators=250, 
                            max_depth=10, 
                            min_samples_split=2, 
                            criterion='entropy',
                            class_weight=None
                           )
rf.fit(X_train,y_train)
rf_train_prob=rf.predict_proba(X_train)[:,1]
rf_train_pred= np.where(rf_train_prob>threshold, 1, 0)
print(metrics.classification_report(rf_train_pred, y_train))
print("")
rf_test_prob=rf.predict_proba(X_test)[:,1]
rf_test_pred=np.where(rf_test_prob>threshold,1,0)
print(metrics.classification_report(rf_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.70      0.82      5053
           1       0.59      0.96      0.73      2229

    accuracy                           0.78      7282
   macro avg       0.78      0.83      0.77      7282
weighted avg       0.86      0.78      0.79      7282


              precision    recall  f1-score   support

           0       0.97      0.78      0.86      1901
           1       0.28      0.79      0.41       209

    accuracy                           0.78      2110
   macro avg       0.63      0.78      0.64      2110
weighted avg       0.90      0.78      0.82      2110



This model is superior to the logistic regression, the weighted average of precision, recall and f1 (weighted by class frequency) are strong and consistent across training and testing set. Note that recall is an important metric in this setting, because we are mainly focused on eliminating false negative (those customers who we think will not churn but then do). False positives, while signal wasted marketing costs, are less of a problem, as they guarantee customers who already would not churn. These metrics suggest a strong and robust model. Therefore, we will use it moving forward to produce the table of churners. 

## 5. Variable Selection

Before we begin to generate the probabilities of churning, we first look to see if we can improve performance through variable selection. The way we did this was to rank the variables by feature importance, and select those with importance>1%. This should further reduce the amount of overfitting.

In [15]:
importance=pd.DataFrame(list(zip(feature_cols, list(rf.feature_importances_))))\
    .sort_values(by=[1], ascending= False)\
    .rename({0:'Variable',1:'Importance'},axis=1)\
    .reset_index(drop=True)
importance

,Variable,Importance
0,Contract_Month-to-month,0.099097
1,Tenure Months,0.099018
2,Dependents,0.097295
3,Contract_Two year,0.090475
4,Monthly Charges,0.085417
5,Total Charges,0.078791
6,Contract_One year,0.058424
7,Online Security,0.054564
8,Tech Support,0.042310
9,Internet_No,0.038623


In [16]:
selected_cols=list(importance.head(18).Variable)

In [33]:
threshold=0.8
rf = RandomForestClassifier(n_estimators=250, 
                            max_depth=10, 
                            min_samples_split=2, 
                            criterion='entropy',
                            class_weight=None
                           )

rf.fit(X_train[selected_cols],y_train)
rf_train_prob=rf.predict_proba(X_train[selected_cols])[:,1]
rf_train_pred= np.where(rf_train_prob>threshold, 1, 0)
print(metrics.classification_report(rf_train_pred, y_train))
print("")
rf_test_prob=rf.predict_proba(X_test[selected_cols])[:,1]
rf_test_pred=np.where(rf_test_prob>threshold,1,0)
print(metrics.classification_report(rf_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      4985
           1       0.60      0.95      0.73      2297

    accuracy                           0.78      7282
   macro avg       0.78      0.83      0.77      7282
weighted avg       0.85      0.78      0.79      7282


              precision    recall  f1-score   support

           0       0.96      0.79      0.87      1845
           1       0.35      0.77      0.48       265

    accuracy                           0.79      2110
   macro avg       0.65      0.78      0.67      2110
weighted avg       0.88      0.79      0.82      2110



This marginally improves the accuracy and recall.

## 6. Evaluating Churn Risk

Now that we have our final model, we can use them to generate probabilities for each individual to churn or not.

In [34]:
swan1=swan.copy()
# Note we only want to predict for those who have not churned yet

swan1['probability_churn']=rf.predict_proba(feature_eng(swan)[selected_cols])[:,1]
top_500_risk=swan1[swan1['Churn Label']=='No']\
        .sort_values(by=['probability_churn'],ascending=False)\
        [['CustomerID','probability_churn']]\
        .head(500)\
        .reset_index(drop=True)
top_500_risk


,CustomerID,probability_churn
0,5542-TBBWB,0.942688
1,7439-DKZTW,0.941681
2,7577-SWIFR,0.938323
3,1452-VOQCH,0.937555
4,5043-TRZWM,0.931759
...,...,...
495,1035-IPQPU,0.656322
496,7529-ZDFXI,0.656015
497,1580-BMCMR,0.655494
498,0869-PAPRP,0.655133


In [35]:
churn_risk=swan1[swan1['Churn Label']=='No']\
        [['CustomerID','probability_churn']]\
        .reset_index(drop=True)
churn_risk

,CustomerID,probability_churn
0,7590-VHVEG,0.663024
1,5575-GNVDE,0.084441
2,7795-CFOCW,0.043035
3,1452-KIOVK,0.238619
4,6713-OKOMC,0.345703
...,...,...
5158,2569-WGERO,0.000235
5159,6840-RESVB,0.039019
5160,2234-XADUH,0.017539
5161,4801-JZAZL,0.141412


In [38]:
top_500_risk.to_csv("top_500_churners.csv")

In [39]:
churn_risk.to_csv("all_churn_risk.csv")